# Práctica 1: Naive Bayes
## Alejandro Benimeli & Rodrigo Juez
### Pareja 07
Primero hemos puesto la ejecución de código con el cual obtenemos los resultados, la memoria y explicaciones están al final del notebook

In [1]:
from Datos import Datos
import EstrategiaParticionado
import Clasificador

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import ShuffleSplit, KFold
from sklearn.naive_bayes import MultinomialNB, CategoricalNB, GaussianNB
import pandas as pd
import numpy as np

err_std_str = "{:.8f} +/- {:.8f}"

# Ejecución Implementación Propia
La función que se encargará de producir los resultados en base al archivo y a la validación elegida.<br>
No tenemos un booleano que diga si es validación simple o cruzada si no eliges la proporción de la validación simple se asume que "numero_particiones" es el KFolds.

In [2]:
# si quieres la validacion simple usa el argumento "proporcion_simple" si no se usará la cruzada
def ejecutar_todo(filename, aplicar_laplace, numero_particiones, proporcion_simple = None):
    dataset = Datos(filename)

    if proporcion_simple != None:
        estrategia = EstrategiaParticionado.ValidacionSimple(proporcion_simple, numero_particiones)
    else:
        estrategia = EstrategiaParticionado.ValidacionCruzada(numero_particiones)


    clasificador = Clasificador.ClasificadorNaiveBayes(aplicar_laplace = aplicar_laplace)
    return clasificador.validacion(estrategia,dataset,clasificador)

### Creamos un dataframe de Pandas vacío, donde luego almacenaremos nuestros resultados

Hemos usado un "MultiIndex" que es un índice de varios niveles que nos ayudará a visualizar la tabla, ya que hay muchas variables comunes

In [3]:
x = ['Error tic-tac-toe', 'Error german']
y = [("Simple", "Si"),
     ("Simple", "No"),
     ("Cruzada", "Si"),
     ("Cruzada", "No")]
names = ["Validación", "Corrección Laplace"]

col_list = pd.MultiIndex.from_tuples(y, names = names)

nuestros_resultados = pd.DataFrame(index=col_list, columns=x, dtype=float)

A continuación solo tenemos que probar todas las combinaciones (Dataset, Validación y Laplace) 2³ y añadirlas a la tabla creada anteriormente.

### Tic Tac Toe

In [4]:
# Validacion Simple con Laplace
err, stdev = ejecutar_todo('../ConjuntosDatos/tic-tac-toe.data', True, 5, proporcion_simple = 0.2)
nuestros_resultados.loc[("Simple", "Si"), "Error tic-tac-toe"] =  err_std_str.format(err, stdev)

# Validacion Simple sin Laplace
err, stdev = ejecutar_todo('../ConjuntosDatos/tic-tac-toe.data', False, 5, proporcion_simple = 0.2)
nuestros_resultados.loc[("Simple", "No"), "Error tic-tac-toe"] =  err_std_str.format(err, stdev)

# Validacion Cruzada con Laplace
err, stdev = ejecutar_todo('../ConjuntosDatos/tic-tac-toe.data', True, 5)
nuestros_resultados.loc[("Cruzada", "Si"), "Error tic-tac-toe"] =  err_std_str.format(err, stdev)

# Validacion Cruzada sin Laplace
err, stdev = ejecutar_todo('../ConjuntosDatos/tic-tac-toe.data', False, 5)
nuestros_resultados.loc[("Cruzada", "No"), "Error tic-tac-toe"] =  err_std_str.format(err, stdev)

### German Data

In [5]:
# Validacion Simple con Laplace
err, stdev = ejecutar_todo('../ConjuntosDatos/german.data', True, 5, proporcion_simple = 0.2)
nuestros_resultados.loc[("Simple", "Si"), "Error german"] =  err_std_str.format(err, stdev)

# Validacion Simple sin Laplace
err, stdev = ejecutar_todo('../ConjuntosDatos/german.data', False, 5, proporcion_simple = 0.2)
nuestros_resultados.loc[("Simple", "No"), "Error german"] =  err_std_str.format(err, stdev)

# Validacion Cruzada con Laplace
err, stdev = ejecutar_todo('../ConjuntosDatos/german.data', True, 5)
nuestros_resultados.loc[("Cruzada", "Si"), "Error german"] =  err_std_str.format(err, stdev)

# Validacion Cruzada sin Laplace
err, stdev = ejecutar_todo('../ConjuntosDatos/german.data', False, 5)
nuestros_resultados.loc[("Cruzada", "No"), "Error german"] =  err_std_str.format(err, stdev)

# Implementación SKlearn
Como para la implementación propia hemos creado una función que ejecuta el fitting y la validación en una sola linea para que nos sea facil de programar.

In [6]:
vs = ShuffleSplit(n_splits=5, test_size=.2)
vc = KFold(n_splits=5, shuffle=True)

In [7]:
def probar_sklearn(X, y, split_strategy, classifierClass, **params):
    errores = []
    for train_idx, test_idx, in split_strategy.split(X):
        classifier = classifierClass(**params)
 
        try:
            classifier.fit(X[train_idx], y[train_idx])
            errores.append(classifier.score(X[test_idx], y[test_idx]))
        except:
            pass

    if len(errores) == 0:
        return 1, 0 # 100% de error
    return 1 - np.mean(errores), np.std(errores)

## Definición de tabla de pandas
Hemos creado la tabla de sklearn lo más parecida a la de nuestra implementación, para que sea facil de comparar entre las dos tablas.

In [8]:
x = ["Error tic-tac-toe", "Error german"]
y = [("CategoricalNB", "Cruzada", "Si"),
     ("CategoricalNB", "Cruzada", "No"),
     ("CategoricalNB", "Simple", "Si"),
     ("CategoricalNB", "Simple", "No"),
     
     ("GaussianNB", "Cruzada", "N/A"),
     ("GaussianNB", "Simple", "N/A"),
     
     ("MultinomialNB", "Cruzada", "Si"),
     ("MultinomialNB", "Cruzada", "No"),
     ("MultinomialNB", "Simple", "Si"),
     ("MultinomialNB", "Simple", "No"),


     ("MultinomialNB | OneHotEncoding", "Cruzada", "Si"),
     ("MultinomialNB | OneHotEncoding", "Cruzada", "No"),
     ("MultinomialNB | OneHotEncoding", "Simple", "Si"),
     ("MultinomialNB | OneHotEncoding", "Simple", "No")]

names = ["Classifier", "Validación", "Laplace"]


col_list = pd.MultiIndex.from_tuples(y, names = names)

sklearn_resultados = pd.DataFrame(index=col_list, columns=x, dtype=float)

## Lectura y Particionado de los datos
Al final hemos usado "ShuffleSplit" y "KFold" que devuelven indices de las particiones en vez de los datos en si.

In [9]:
dataset_german=Datos('../ConjuntosDatos/german.data')
dataset_tictactoe=Datos('../ConjuntosDatos/tic-tac-toe.data')

X_german = dataset_german.datos.loc[:, dataset_german.datos.columns != "Class"].to_numpy()
y_german = dataset_german.datos["Class"].to_numpy()

X_tictactoe = dataset_tictactoe.datos.loc[:, dataset_tictactoe.datos.columns != "Class"].to_numpy()
y_tictactoe = dataset_tictactoe.datos["Class"].to_numpy()

vs = ShuffleSplit(n_splits=5, test_size=.2)
vc = KFold(n_splits=5, shuffle=True)

Si observa la celda de imports podrá ver que hay una string llamada "err_std_str", esta se encarga de formatear el error y la std para su visualización en la tabla, y como recibimos de la función que ejecuta el fitting y la validación el err y std en el mismo orden podemos ponerlo como argumento y descomprimir la tupla con un asterisco.

Igualmente añadimos los resultados a la tabla, que tiene un formato muy similar a la que usamos para nuestra implementación.

## GaussianNB

In [10]:
print("GaussianNB - Simple")
errgerman = err_std_str.format(*probar_sklearn(X_german, y_german, vs, GaussianNB))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe, y_tictactoe, vs, GaussianNB))
sklearn_resultados.loc[('GaussianNB', 'Simple', 'N/A'), 'Error german'] = errgerman
sklearn_resultados.loc[('GaussianNB', 'Simple', 'N/A'), 'Error tic-tac-toe'] = errtictactoe


print("GaussianNB - Cruzada")
errgerman = err_std_str.format(*probar_sklearn(X_german, y_german, vc, GaussianNB))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe, y_tictactoe, vc, GaussianNB))
sklearn_resultados.loc[('GaussianNB', 'Cruzada', 'N/A'), 'Error german'] = errgerman
sklearn_resultados.loc[('GaussianNB', 'Cruzada', 'N/A'), 'Error tic-tac-toe'] = errtictactoe

GaussianNB - Simple
GaussianNB - Cruzada


## CategoricalNB

In [11]:
print("CategoricalNB - Simple - No Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german, y_german, vs, CategoricalNB, alpha=1e-09, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe, y_tictactoe, vs, CategoricalNB, alpha=1e-09, fit_prior=True))
sklearn_resultados.loc[('CategoricalNB', 'Simple', 'No'), 'Error german'] = errgerman
sklearn_resultados.loc[('CategoricalNB', 'Simple', 'No'), 'Error tic-tac-toe'] = errtictactoe


print("CategoricalNB - Simple - Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german, y_german, vs, CategoricalNB, alpha=1, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe, y_tictactoe, vs, CategoricalNB, alpha=1, fit_prior=True))
sklearn_resultados.loc[('CategoricalNB', 'Simple', 'Si'), 'Error german'] = errgerman
sklearn_resultados.loc[('CategoricalNB', 'Simple', 'Si'), 'Error tic-tac-toe'] = errtictactoe


print("CategoricalNB - Cruzada - No Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german, y_german, vc, CategoricalNB, alpha=1e-09, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe, y_tictactoe, vc, CategoricalNB, alpha=1e-09, fit_prior=True))
sklearn_resultados.loc[('CategoricalNB', 'Cruzada', 'No'), 'Error german'] = errgerman
sklearn_resultados.loc[('CategoricalNB', 'Cruzada', 'No'), 'Error tic-tac-toe'] = errtictactoe


print("CategoricalNB - Cruzada - Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german, y_german, vc, CategoricalNB, alpha=1, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe, y_tictactoe, vc, CategoricalNB, alpha=1, fit_prior=True))
sklearn_resultados.loc[('CategoricalNB', 'Cruzada', 'Si'), 'Error german'] = errgerman
sklearn_resultados.loc[('CategoricalNB', 'Cruzada', 'Si'), 'Error tic-tac-toe'] = errtictactoe

CategoricalNB - Simple - No Laplace
CategoricalNB - Simple - Laplace
CategoricalNB - Cruzada - No Laplace
CategoricalNB - Cruzada - Laplace


## MultinomialNB

In [12]:
print("MultinomialNB - Simple - No Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german, y_german, vs, MultinomialNB, alpha=1e-09, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe, y_tictactoe, vs, MultinomialNB, alpha=1e-09, fit_prior=True))
sklearn_resultados.loc[('MultinomialNB', 'Simple', 'No'), 'Error german'] = errgerman
sklearn_resultados.loc[('MultinomialNB', 'Simple', 'No'), 'Error tic-tac-toe'] = errtictactoe



print("MultinomialNB - Simple - Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german, y_german, vs, MultinomialNB, alpha=1, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe, y_tictactoe, vs, MultinomialNB, alpha=1, fit_prior=True))
sklearn_resultados.loc[('MultinomialNB', 'Simple', 'Si'), 'Error german'] = errgerman
sklearn_resultados.loc[('MultinomialNB', 'Simple', 'Si'), 'Error tic-tac-toe'] = errtictactoe



print("MultinomialNB - Cruzada - No Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german, y_german, vc, MultinomialNB, alpha=1e-09, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe, y_tictactoe, vc, MultinomialNB, alpha=1e-09, fit_prior=True))
sklearn_resultados.loc[('MultinomialNB', 'Cruzada', 'No'), 'Error german'] = errgerman
sklearn_resultados.loc[('MultinomialNB', 'Cruzada', 'No'), 'Error tic-tac-toe'] = errtictactoe




print("MultinomialNB - Cruzada - Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german, y_german, vc, MultinomialNB, alpha=1, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe, y_tictactoe, vc, MultinomialNB, alpha=1, fit_prior=True))
sklearn_resultados.loc[('MultinomialNB', 'Cruzada', 'Si'), 'Error german'] = errgerman
sklearn_resultados.loc[('MultinomialNB', 'Cruzada', 'Si'), 'Error tic-tac-toe'] = errtictactoe


MultinomialNB - Simple - No Laplace
MultinomialNB - Simple - Laplace
MultinomialNB - Cruzada - No Laplace
MultinomialNB - Cruzada - Laplace


## MultinomialNB | OneHotEncoding

In [13]:
encoder = OneHotEncoder()
X_german_onehot = encoder.fit_transform(X_german)
X_tictactoe_onehot = encoder.fit_transform(X_tictactoe)

print("MultinomialNB OneHot - Simple - No Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german_onehot, y_german, vs, MultinomialNB, alpha=1e-09, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe_onehot, y_tictactoe, vs, MultinomialNB, alpha=1e-09, fit_prior=True))
sklearn_resultados.loc[('MultinomialNB | OneHotEncoding', 'Simple', 'No'), 'Error german'] = errgerman
sklearn_resultados.loc[('MultinomialNB | OneHotEncoding', 'Simple', 'No'), 'Error tic-tac-toe'] = errtictactoe



print("MultinomialNB OneHot - Simple - Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german_onehot, y_german, vs, MultinomialNB, alpha=1, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe_onehot, y_tictactoe, vs, MultinomialNB, alpha=1, fit_prior=True))
sklearn_resultados.loc[('MultinomialNB | OneHotEncoding', 'Simple', 'Si'), 'Error german'] = errgerman
sklearn_resultados.loc[('MultinomialNB | OneHotEncoding', 'Simple', 'Si'), 'Error tic-tac-toe'] = errtictactoe



print("MultinomialNB OneHot - Cruzada - No Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german_onehot, y_german, vc, MultinomialNB, alpha=1e-09, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe_onehot, y_tictactoe, vc, MultinomialNB, alpha=1e-09, fit_prior=True))
sklearn_resultados.loc[('MultinomialNB | OneHotEncoding', 'Cruzada', 'No'), 'Error german'] = errgerman
sklearn_resultados.loc[('MultinomialNB | OneHotEncoding', 'Cruzada', 'No'), 'Error tic-tac-toe'] = errtictactoe



print("MultinomialNB OneHot - Cruzada - Laplace")
errgerman = err_std_str.format(*probar_sklearn(X_german_onehot, y_german, vc, MultinomialNB, alpha=1, fit_prior=True))
errtictactoe = err_std_str.format(*probar_sklearn(X_tictactoe_onehot, y_tictactoe, vc, MultinomialNB, alpha=1, fit_prior=True))
sklearn_resultados.loc[('MultinomialNB | OneHotEncoding', 'Cruzada', 'Si'), 'Error german'] = errgerman
sklearn_resultados.loc[('MultinomialNB | OneHotEncoding', 'Cruzada', 'Si'), 'Error tic-tac-toe'] = errtictactoe

MultinomialNB OneHot - Simple - No Laplace
MultinomialNB OneHot - Simple - Laplace
MultinomialNB OneHot - Cruzada - No Laplace
MultinomialNB OneHot - Cruzada - Laplace


# Memoria

# Apartado 1

In [14]:
nuestros_resultados

Error tic-tac-toe  \
Validación Corrección Laplace                              
Simple     Si                  0.29633508 +/- 0.01175390   
           No                  0.30785340 +/- 0.03023985   
Cruzada    Si                  0.29235929 +/- 0.04889966   
           No                  0.30274324 +/- 0.02151151   

                                            Error german  
Validación Corrección Laplace                             
Simple     Si                  0.25400000 +/- 0.01854724  
           No                  0.25400000 +/- 0.01392839  
Cruzada    Si                  0.25300000 +/- 0.01720465  
           No                  0.25100000 +/- 0.02083267

Lo primero que apreciamos, es que nuestro clasificador obtiene mejores resultados con el dataset de german data en general. Posiblemente se deba a que tic-tac-toe tiene muchos menos atributos y son todos nominales.

También vemos que tanto la validación simple como cruzada ofrecen un rendimiento muy similar, teniendo la validación cruzada una desviación algo mayor en general, pensamos que podría ser porque validación cruzada siempre se asegura que en las n iteraciones nunca se repita el test ya que va fold por fold asegurándose de que todos los datos pasan por el test alguna vez.

Es decir si hay alguna fila que pueda provocar que suba mucho el error KFold se asegura que en alguna partición esté incluida, mientras que, validación simple es completamente aleatorio y a lo mejor algunas iteraciones coinciden o hay algunas líneas, que a lo mejor provocan que aumente el error, no aparecen.

En cuanto al efecto Laplace, suaviza un poco las verosimilitudes añadiendo un “1” (u otro número, aunque luego explicamos por qué usamos el 1) a cada valor del atributo. Resuelve el problema de tener una probabilidad de cero si algún dato no aparece en el dataset de train, ya que ahora hay un 1 en esa casilla en vez de un cero. Al usar valores muy grandes al suavizar con Laplace, hacemos que las verosimilitudes se acerquen más a 1/número de valores del atributo. Por lo tanto es recomendable usar valores más pequeños (1 en nuestro caso)

En nuestro caso concreto Laplace mejora ligeramente el resultado o lo mantiene igual, aunque en pocas ocasiones puede empeorar el resultado.

# Apartado 2

In [15]:
sklearn_resultados

Error tic-tac-toe  \
Classifier                     Validación Laplace                              
CategoricalNB                  Cruzada    Si       0.29226658 +/- 0.02098081   
                                          No       0.30579188 +/- 0.03161378   
                               Simple     Si       0.28229167 +/- 0.02291667   
                                          No       0.30520833 +/- 0.02072890   
GaussianNB                     Cruzada    N/A      0.29121946 +/- 0.01569761   
                               Simple     N/A      0.30729167 +/- 0.02696287   
MultinomialNB                  Cruzada    Si       0.34552792 +/- 0.02519938   
                                          No       0.34341187 +/- 0.01217281   
                               Simple     Si       0.34583333 +/- 0.02993772   
                                          No       0.32708333 +/- 0.04651482   
MultinomialNB | OneHotEncoding Cruzada    Si       0.30165249 +/- 0.02974940   
                                          No       0.30071990 +/- 0.04869375   
                               Simple     Si       0.29895833 +/- 0.02149976   
                                          No       0.28020833 +/- 0.03644345   

                                                                Error german  
Classifier                     Validación Laplace                             
CategoricalNB                  Cruzada    Si       0.26833333 +/- 0.01649916  
                                          No       0.32500000 +/- 0.03188521  
                               Simple     Si       0.27625000 +/- 0.01138804  
                                          No       0.31000000 +/- 0.01500000  
GaussianNB                     Cruzada    N/A      0.26600000 +/- 0.01933908  
                               Simple     N/A      0.26400000 +/- 0.01240967  
MultinomialNB                  Cruzada    Si       0.35800000 +/- 0.03203123  
                                          No       0.35900000 +/- 0.04127953  
                               Simple     Si       0.37900000 +/- 0.01240967  
                                          No       0.34000000 +/- 0.02024846  
MultinomialNB | OneHotEncoding Cruzada    Si       0.27000000 +/- 0.01673320  
                                          No       0.30900000 +/- 0.01827567  
                               Simple     Si       0.26700000 +/- 0.02619160  
                                          No       0.29500000 +/- 0.01702939

<b>CategoricalNB:</b><br>
* alphafloat es la aplicación de Laplace, nosotros solo usamos 1 o 0 porque en nuestra implementación propia o sumamos 1 o no sumamos nada, pero en sklearn te deja usar valores intermedios.

* fit_prior: Lo seteamos a True porque se corresponde a lo que hacemos en nuestra implementación propia, es decir calcular los priores para cada valor de la clase a partir de los datos. Si estuviese a False entonces usaría un prior uniforme, esto quiere decir que se usa siempre el mismo prior para hacer smoothing.

Cuando implementamos CategoricalNB nos daba error al ejecutar porque algunos valores que aparecían en el test no aparecían en la fase de train, esto es porque usa una distribución Categórica.
Además CategoricalNB trata los atributos como discretos.<br><br><br>


<b>GaussianNB:</b><br>
No usamos ningún atributo, ya que como usa una distribución gaussiana y asume que todos los atributos son continuos, no se puede aplicar Laplace. Además el resto de atributos no entendíamos que hacían ni podíamos relacionarlos con la teoría.<br><br>

<b>MultinomialNB:</b><br>
* alphafloat es la aplicación de Laplace, que ya hemos explicado en CategoricalNB.
* fit_prior: Lo seteamos a True porque se corresponde a lo que hacemos en nuestra implementación propia y ya hemos explicado que hace en CategoricalNB.

Al igual que CategoricalNB, MultinomialNB es para features discretas.<br><br>


<b>MultinomialNB | OneHotEncoding:</b><br>
OneHotEncoding consiste en separar atributos existentes en varios atributos de 1 y 0, es decir, si tenemos una clase con valores posibles 1,2,3,4, tendremos 4 atributos nuevos (1,2,3,4) con valores 1 o 0 dependiendo de si esa fila tiene ese valor o no, con lo que dejamos de asumir que las features tienen un orden.

Se puede observar como el OneHotEncoding mejora mucho la tasa de error de MultinomialNB.
Si revisamos los dataset podemos ver como german.data tiene, en los atributos nominales, una gran cantidad de valores posibles, esta puede ser la razón por la que para german la mejora es mucho mayor que para tic-tac-toe.

Todos los clasificadores salvo MultonomialNB sin OneHotEncoding funcionan similar en el dataset de tic-tac-toe, siendo MultinomialNB con OneHotEncoding el que mejor rinde con una tasa del 28% de error.

Respecto al dataset german.data GaussianNB funciona bastante mejor que los demás, esto puede ser porque, aún teniendo atributos nominales, el número de atributos continuos es mayor que en tic-tac-toe y GaussianNB funciona mejor en ellos, dado que sigan una distribución normal.

De todos los clasificadores, el único de ellos que rinde mejor con german es GaussianNB, ya que ese dataset tiene varios atributos continuos. El resto rinden mejor con tic-tac-toe, ya que están diseñados para atributos discretos y tic-tac-toe solo tiene atributos discretos.


# Apartado 3

Podemos ver que para ambos datasets, nuestra implementación es la que mejor precisión da. Esto se debe a que los algoritmos de SKLearn tratan todos los datos como si fuesen discretos o continuos (pero no ambos a la vez), mientras que nosotros sí distinguimos y calculamos las probabilidades de cada atributo de diferente forma dependiendo de si es continuo o discreto. 

A pesar de eso, una ventaja que tiene SKLearn es la velocidad de procesamiento ya que está programado con librerias en C y optimizaciones, mientras que nuestro código es python puro que es mucho más lento.

Hemos usado la librería de time para medir el tiempo y compararlo y nuestra ejecución tarda de media 1,9 segundos mientras que sklearn terminaba en 0,00300s.